In [ ]:
import random
import osmnx as ox
import networkx as nx
from geopy.distance import geodesic
import folium

class APIGateway:
    """Simula el enrutamiento de solicitudes en el sistema."""
    @staticmethod
    def route_request(service, *args, **kwargs):
        return service(*args, **kwargs)

class AuthService:
    """Simula la autenticación de usuarios."""
    @staticmethod
    def authenticate(user_id):
        return user_id in range(1, 100)

class InventoryService:
    """Servicio ficticio para cumplir con la arquitectura."""
    def get_inventory(self):
        return {}

class OrderService:
    """Servicio ficticio para cumplir con la arquitectura."""
    def create_order(self):
        return "Order Created"

class TrackingService:
    """Simula el rastreo de las rutas reales de los vendedores."""
    def __init__(self):
        self.graph = ox.graph_from_place("Medellín, Colombia", network_type="drive")

    def get_route(self, origin, destination):
        orig_node = ox.distance.nearest_nodes(self.graph, origin[1], origin[0])
        dest_node = ox.distance.nearest_nodes(self.graph, destination[1], destination[0])
        route = nx.shortest_path(self.graph, orig_node, dest_node, weight="length")
        return [(self.graph.nodes[n]['y'], self.graph.nodes[n]['x']) for n in route]

    def get_real_routes(self, num_valid=30, num_invalid=5):
        nodes = list(self.graph.nodes())
        valid_routes = []
        for _ in range(num_valid):
            start, end = random.sample(nodes, 2)
            route = self.get_route((self.graph.nodes[start]['y'], self.graph.nodes[start]['x']),
                                   (self.graph.nodes[end]['y'], self.graph.nodes[end]['x']))
            valid_routes.append(route)
        real_routes = [route.copy() for route in valid_routes]
        for _ in range(num_invalid):
            idx = random.randint(0, len(real_routes) - 1)
            start, end = random.sample(nodes, 2)
            real_routes[idx] = self.get_route((self.graph.nodes[start]['y'], self.graph.nodes[start]['x']),
                                              (self.graph.nodes[end]['y'], self.graph.nodes[end]['x']))
        return {"valid_routes": valid_routes, "real_routes": real_routes}

class RouteService:
    """Simula la planificación y optimización de rutas."""
    @staticmethod
    def plan_route(user_id):
        return [(6.2442, -75.5812), (6.2471, -75.5828), (6.2499, -75.5845)]

class VerificationService:
    """Compara las rutas esperadas con las reales para detectar discrepancias."""
    @staticmethod
    def detect_inconsistencies(reported_route, actual_route, threshold_km=0.5):
        for r, a in zip(reported_route, actual_route):
            if geodesic(r, a).km > threshold_km:
                return True  # Hay discrepancia
        return False  # No hay discrepancia

    def validate_routes(self, reported_routes, real_routes):
        discrepancies = [self.detect_inconsistencies(r, a) for r, a in zip(reported_routes, real_routes)]
        return discrepancies

def plot_routes(routes, discrepancies):
    """Visualiza solo las rutas de real_routes que tienen discrepancias con valid_routes en un mapa de Medellín."""
    map_mde = folium.Map(location=[6.25, -75.58], zoom_start=13)
    # Agregar rutas válidas en azul
    for route in routes["valid_routes"]:
        folium.PolyLine(route, color='blue', weight=3, opacity=0.8, tooltip="Ruta Válida").add_to(map_mde)
    
    for idx, (route, discrepancy) in enumerate(zip(routes["real_routes"], discrepancies)):
        if discrepancy:  
            folium.PolyLine(route, color='red', weight=3, opacity=0.8, tooltip="Ruta Realizada con Discrepancia").add_to(map_mde)

    map_mde.save("rutas_medellin_discrepancias.html")
    print("Mapa guardado como rutas_medellin_discrepancias.html")

# Simulación del experimento
def run_experiment():
    tracking_service = TrackingService()
    verification_service = VerificationService()
    routes = tracking_service.get_real_routes()
    discrepancies = verification_service.validate_routes(routes["valid_routes"], routes["real_routes"])
    detected_discrepancies = sum(discrepancies)
    total_routes = len(discrepancies)
    print(f"Discrepancias detectadas: {detected_discrepancies} de {total_routes}")
    
    plot_routes(routes, discrepancies)
    return detected_discrepancies, total_routes

if __name__ == "__main__":
    run_experiment()

Discrepancias detectadas: 5 de 30
Mapa guardado como rutas_medellin_discrepancias.html
